In [1]:
import pandas as pd
import numpy as np
# import nilearn"

In [18]:
data = pd.read_csv('noraml_results.csv',low_memory=False)
# ids = data['SubjectID']

In [19]:
# data = data.drop(['SubjectID'],axis=1)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208 entries, 0 to 1207
Columns: 546 entries, SubjectID to T2_ED_LBP_Radius-1_Bins-20_LBP
dtypes: float64(334), int64(136), object(76)
memory usage: 5.0+ MB


In [26]:

# np.sum(data_numeric['FLAIR_ED_Intensity_CoefficientOfVariation'].astype(float).isnull())
data_numeric = data.apply(pd.to_numeric,errors='coerce')
data_na = data_numeric[data_numeric.isna().any(axis=1)]

# data_na.drop([293,743,904],axis=0)
data_na





,SubjectID,FLAIR_ED_Intensity_CoefficientOfVariation,FLAIR_ED_Intensity_Energy,FLAIR_ED_Intensity_InterQuartileRange,FLAIR_ED_Intensity_Kurtosis,FLAIR_ED_Intensity_Maximum,FLAIR_ED_Intensity_Mean,FLAIR_ED_Intensity_MeanAbsoluteDeviation,FLAIR_ED_Intensity_Median,FLAIR_ED_Intensity_MedianAbsoluteDeviation,...,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeMean,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeNonUniformity,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeNoneUniformityNormalized,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeVariance,T2_ED_NGTDM_Busyness,T2_ED_NGTDM_Coarsness,T2_ED_NGTDM_Complexity,T2_ED_NGTDM_Contrast,T2_ED_NGTDM_Strength,T2_ED_LBP_Radius-1_Bins-20_LBP
9,14,0.144956,4.546696e+04,30.747772,0.250000,165.364304,149.990418,0.000000e+00,149.990418,0.000000,...,1.000000,2.000000,1.000000,0.000000,0.552632,0.095238,199.500000,947.625000,34.380952,30.064860
100,140,0.030319,1.500581e+06,6.000000,2.807781,170.538467,158.073077,-5.684342e-15,157.730774,0.342303,...,1.132075,42.169811,0.795657,0.152367,0.253918,0.024048,1523.005645,1.244576,3.517641,863.496703
176,258,0.163524,6.730304e+04,18.422050,1.270947,142.205322,114.798476,-5.684342e-15,109.239540,5.558936,...,1.250000,2.500000,0.625000,0.187500,0.500000,0.063291,319.466667,67.466667,12.007273,70.456887
234,338,0.014886,1.571243e+05,4.467148,1.066010,164.540146,161.810219,0.000000e+00,161.934311,-0.124092,...,1.200000,3.400000,0.680000,0.160000,0.380952,0.093750,317.888889,29.948148,14.346154,90.171584
293,425,NaN,2.377845e+03,0.000000,NaN,48.763157,48.763157,0.000000e+00,48.763157,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,15.032430
305,444,0.087228,5.655807e+04,7.083336,1.831951,91.901711,79.006412,3.157968e-15,75.918808,3.087604,...,1.000000,9.000000,1.000000,0.000000,0.379397,0.059603,456.566667,28.498765,13.892720,135.282465
743,1180,NaN,1.272891e+04,0.000000,NaN,112.822472,112.822472,0.000000e+00,112.822472,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,15.032430
904,1344,NaN,3.459157e+04,0.000000,NaN,185.988098,185.988098,0.000000e+00,185.988098,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,15.032430
1120,1568,0.106352,2.146930e+05,15.738022,2.803975,199.955627,163.014196,0.000000e+00,160.067993,2.946203,...,1.000000,8.000000,1.000000,0.000000,0.309524,0.102564,321.500000,17.375000,15.200000,120.201633


In [27]:
data_clean = data_numeric.dropna()
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199 entries, 0 to 1207
Columns: 546 entries, SubjectID to T2_ED_LBP_Radius-1_Bins-20_LBP
dtypes: float64(410), int64(136)
memory usage: 5.0 MB


In [28]:
Ids_clean = data_clean['SubjectID']
data_clean.drop(['SubjectID'],axis=1,inplace=True)

/var/folders/v8/wgs84bvx4_z63x9x9m3sq2kc0000gn/T/ipykernel_7474/4113870824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.drop(['SubjectID'],axis=1,inplace=True)


In [29]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
X_pca = pca.fit_transform(data_clean)
X_pca.shape

(1199, 2)

In [30]:
data_clean.describe()

,FLAIR_ED_Intensity_CoefficientOfVariation,FLAIR_ED_Intensity_Energy,FLAIR_ED_Intensity_InterQuartileRange,FLAIR_ED_Intensity_Kurtosis,FLAIR_ED_Intensity_Maximum,FLAIR_ED_Intensity_Mean,FLAIR_ED_Intensity_MeanAbsoluteDeviation,FLAIR_ED_Intensity_Median,FLAIR_ED_Intensity_MedianAbsoluteDeviation,FLAIR_ED_Intensity_Minimum,...,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeMean,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeNonUniformity,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeNoneUniformityNormalized,T2_ED_GLSZM_Bins-20_Radius-1_ZoneSizeVariance,T2_ED_NGTDM_Busyness,T2_ED_NGTDM_Coarsness,T2_ED_NGTDM_Complexity,T2_ED_NGTDM_Contrast,T2_ED_NGTDM_Strength,T2_ED_LBP_Radius-1_Bins-20_LBP
count,1199.000000,1.199000e+03,1199.000000,1199.000000,1199.000000,1199.000000,1.199000e+03,1199.000000,1199.000000,1199.000000,...,1199.000000,1199.000000,1199.000000,1.199000e+03,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1.199000e+03
mean,0.177393,2.788463e+08,28.227742,4.106583,203.710118,130.633555,1.086488e-14,130.736884,-0.103329,53.261240,...,20.315380,159.341054,0.265491,7.458329e+04,75.264390,0.001429,2105.862708,0.776669,0.227368,1.301169e+05
std,0.106284,4.624571e+08,17.023168,2.823798,38.596151,34.396343,1.684440e-12,36.125748,4.532596,40.109458,...,22.100887,144.147530,0.071981,3.130635e+05,112.526819,0.005588,776.021939,0.950911,0.828718,1.729391e+05
min,0.023032,2.774825e+05,4.767082,1.382338,46.064514,24.786431,-1.581604e-11,16.642105,-34.007919,0.000000,...,1.130435,4.444444,0.127105,1.134216e-01,0.178870,0.000003,284.604279,0.062522,0.000695,1.556113e+02
25%,0.108103,3.765783e+07,16.948475,2.800031,176.987129,107.583285,-2.199055e-13,106.656532,-1.652268,18.905645,...,7.867821,57.553633,0.220451,6.905632e+02,11.163657,0.000042,1572.574407,0.440125,0.008473,2.337438e+04
50%,0.156493,1.219505e+08,24.118378,3.446138,208.298538,127.133757,-2.561957e-15,127.062866,0.035359,48.036270,...,14.631893,119.649484,0.252169,5.721402e+03,37.037973,0.000120,1910.085217,0.637600,0.021977,7.317508e+04
75%,0.218871,3.398300e+08,34.073505,4.553877,235.774460,154.697804,1.935651e-13,156.366913,1.820806,80.806946,...,24.983548,211.650124,0.291940,3.371772e+04,92.430472,0.000429,2429.414300,0.916014,0.076311,1.708382e+05
max,1.054095,4.685167e+09,163.679184,57.406124,272.704315,229.664194,2.599775e-11,228.763855,32.019490,204.739380,...,355.101124,1532.664084,0.773157,5.783924e+06,1052.830628,0.072072,5478.541734,24.580977,10.253666,1.509632e+06


In [36]:
from sklearn.decomposition import KernelPCA
kPca = KernelPCA(n_components=10,kernel='poly')
X_kpca = kPca.fit_transform(data_clean)
X_kpca.shape

(1199, 10)

In [ ]:
import nibabel as nib


flair_0 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00000/BraTS2021_00000_flair.nii.gz')
t1_0 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00000/BraTS2021_00000_t1.nii.gz')
t1ce_0 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00000/BraTS2021_00000_t1ce.nii.gz')
t2_0 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00000/BraTS2021_00000_t2.nii.gz')
mask_0  = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00000/BraTS2021_00000_seg.nii.gz')

flair_2 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00002/BraTS2021_00002_flair.nii.gz')
t1_2 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00002/BraTS2021_00002_t1.nii.gz')
t1ce_2 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00002/BraTS2021_00002_t1ce.nii.gz')
t2_2 = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00002/BraTS2021_00002_t2.nii.gz')
mask_2  = nib.load('/home/aditya/Documents/HandsonML/Brats_demo/BraTS2021_00002/BraTS2021_00002_seg.nii.gz')

In [ ]:
flair_0.shape, mask_0.shape

In [ ]:
imarray = flair_0.get_fdata()


In [ ]:
np.amax(imarray),np.amin(imarray)

In [ ]:
def Image_scaler(image):
    max,min = np.amax(image),np.amin(image)
    scaled = np.subtract(image,min)
    


In [ ]:
from sklearn.preprocessing import MinMaxScaler
# fslice = flair.slicer[:,:,-2:-1]
scaler = MinMaxScaler(feature_range=(0,255))
scaled = scaler.fit_transform(imarray)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# plt.figure(figsize=(10,20))
plt.imshow(farray[:,100,:])

In [ ]:
plt.imshow(marray[:,100,:])

In [ ]:
masked = np.multiply(farray,marray)

In [ ]:
plt.imshow(masked[:,100,:])


In [ ]:
flat = np.ndarray.flatten(masked).reshape(1,-1)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99999)
X_red_train = pca.fit_transform(flat)

In [ ]:
X_red_train.shape